#Project Proposal: SBIR award prediction

#### Reminder from first proposal --> High level view:
The Small Business Innovation Research (SBIR,  https://www.sbir.gov/) program relies on an award-based system to trigger high-tech innovation in small US companies. Many agencies (Departement of Agriculture, of Defense, ...) contribute to this fund. Two main phases structure the SBIR program: phase I (approx 150 000 dollars, 6 months) and phase II (approx 1 000 000 dollars for 2 years). To apply to phase II, companies must be phase I awardees. 

The predictive question I want to answer is: will your project make it to phase II?  
The interpretative question I would like to address is: what features make you likely to be successful in applying to phase II?

####Aim of this document --> establish the presence of a signal in the data:
This document follows the pipeline I developped to evaluate a simple predictive model (Logistic Regression) on a small dataset. In the end, this exploratory work shows that the simple predictor outperforms a baseline predictor (one that randomly predicts success or failure with the baseline probability of the training set). Further ideas to increase predictive power are also listed.

Data and code I used for this proposal can be found on GitHub, at https://github.com/AnnaVM/SBIR_Project

In [143]:
import pandas as pd
from sklearn.cross_validation import KFold

#from code in the GitHub Repro (https://github.com/AnnaVM/SBIR_Project)
from prepare_data import subset_data
from model import Model

###Loading a subset of the data

For this proposal, I worked only on a subset of my data (data from years 2012 to 2015, with one specific agency, the Department of Defense 'dod'). 
- A first converter allowed me to go from .xlsx files to .csv files. (not shown here, available on the GitHub Repo)
- A second function allows me to work on the subset (code and csv files are available on the GitHub Repo)

In [243]:
df = subset_data('dod', 2012, '/Users/AnnaVMS/Desktop/test2')
#update path according to download

['2015-12-14_award_export_10001TO15000.csv', '2015-12-14_award_export_1TO5000.csv', '2015-12-14_award_export_5001TO10000.csv']


prepare_data.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  .apply(lambda x: is_in_phase_I_to_II(x, phase_I_to_II))


In [233]:
print 'number of phase I projects considered: ',len(df.to_phase_II)

number of phase I projects considered:  1016


In [234]:
print 'number of phase I projects with successful application to phase II: ',sum(df.to_phase_II)

number of phase I projects with successful application to phase II:  305


In [236]:
print 'number of phase I projects without a phase II: ',len(df.to_phase_II)-sum(df.to_phase_II)

number of phase I projects without a phase II:  711


In [239]:
#quick overview of the dataset
print 'information in the database: \n',df.columns

information in the database: 
Index([u'index', u'Company', u'Award Title', u'Agency', u'Branch', u'Phase',
       u'Program', u'Agency Tracking #', u'Contract', u'Award Start Date',
       u'Award Close Date', u'Solicitation #', u'Solicitation Year',
       u'Topic Code', u'Award Year', u'Award Amount', u'DUNS',
       u'Hubzone Owned', u'Socially and Economically Disadvantaged',
       u'Woman Owned', u'# Employees', u'Company Website', u'Address1',
       u'Address2', u'City', u'State', u'Zip', u'Contact Name',
       u'Contact Title', u'Contact Phone', u'Contact Email', u'PI Name',
       u'PI Title', u'PI Phone', u'PI Email', u'RI Name', u'RI POC Name',
       u'RI POC Phone', u'Research Keywords', u'Abstract', u'to_phase_II'],
      dtype='object')


- The last column 'to_phase_II' gives information on whether a phase II was obtained. It will be used as the label to predict in my model
- Features can be choosen in the rest of the columns (here for instance: 'Solicitation Year', 'Award Amount', 'Hubzone Owned', 'Socially and Economically Disadvantaged', 'Woman Owned', '# Employees') or engineered (from 'Abstract', I made 'Abstract Length'--simple character count-- and 'Topic of Abstract' --through NMF topic modeling)

###defining a first model, based on this subset of data

In [221]:
#defining a training and testing set (through indices)
kf = KFold(1016,5, shuffle=True)
kf_iterator = kf.__iter__()
train_index, test_index = kf_iterator.next()

####The Pipeline:
1- Processing the text of the Abstract of submissions
- tfidf vectorization of the text (5000 words in the dictionary, default tokenization of sklearn)
- topic modeling: NMF (5 components, no optimization done here)

2- Preparing data for model
- using dummy variables for topics
- mapping Y/N to 1/0
- getting the abstract length in characters

3- Running the Logistic Regression
- Standard scaling of the data
- Logistic Regression (lasso regularization)
--> outputs the score

In [224]:
model_test = Model(df, train_index, test_index)

In [240]:
#step 1
model_test.process_text('Abstract')

In [241]:
#step 2
model_test.prepare_LogReg()

In [242]:
#step 3
model_test.perform_LogReg()

(0.73399014778325122, 0.58584468270066636, 0.73529411764705888)

- The first score is the accuracy of the Logistic Classifier on the training set;
- the second score is the accuracy of the baseline predictor (one that randomly predicts success or failure with the baseline probability of the training set),
- the third score gives the accuracy of the Logistic Classifier on the test set.

####Some information on the models (coefficients from LogReg and topics from NMF)

In [231]:
#the coefficients in the Logisitic Regression (lasso regularized)
pd.DataFrame( {'criterion:': model_test.list_columns[:-1], 
               'coeffs': model_test.model_LogReg.coef_[0]} )

,coeffs,criterion:
0,0.009701,topic 4
1,-0.074706,topic 3
2,0.050049,topic 2
3,-0.162837,topic 1
4,0.000000,topic 0
5,0.135534,Solicitation Year
6,0.667513,Award Amount
7,0.050143,Hubzone Owned as_int
8,-0.139014,Socially and Economically Disadvantaged as_int
9,0.084231,Woman Owned as_int


In [232]:
#topics defined with NMF:
for i in xrange(5):
    print model_test.get_top_n_words_for_topic(i)

[u'high' u'power' u'materials' u'laser' u'phase' u'technology' u'design'
 u'low' u'applications' u'energy']
[u'data' u'information' u'network' u'security' u'analysis' u'cyber'
 u'networks' u'quot' u'satellite' u'software']
[u'model' u'models' u'plume' u'modeling' u'simulation' u'software'
 u'effects' u'missile' u'3d' u'damage']
[u'training' u'learning' u'rpa' u'adaptive' u'performance' u'game'
 u'trainee' u'based' u'tool' u'simulation']
[u'sensor' u'radar' u'sensors' u'target' u'tracking' u'space' u'detection'
 u'objects' u'surveillance' u'processing']


###Further Work

Other, easy but time-consuming developments
- adding other features (State...)
- vectorizing and topic modeling for other text inputs (Title, Keywords, Contact Title...)
- adding further company information
- tuning regularization on Logistic Regression
- running other models (getting feature importance with Random Forest for interpretation, trying to increase predictive power with SVM)

Other, harder, routes:
- developping a sentiment analysis (very technical, very assured, very detail-oriented, very goal-oriented)

###Bibliography
A paper featured in Kaggle, as a source inspiration
- http://cs.stanford.edu/~althoff/raop-dataset/altruistic_requests_icwsm.pdf